In [1]:
%load_ext autotime

import os, math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm, trange

In [2]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, BatchNormalization, Bidirectional
from sklearn.metrics import mean_squared_error, mean_absolute_error

def model(test_data, train_data):
    test_time = test_data['time']
    test_gl_value = test_data['gl_value']
    
    test_data.drop(columns = ['time'], inplace = True)
    train_data.rename(columns = {'bolus_type_normal dual':'bolus_type_normal_dual', 'bolus_type_square dual':'bolus_type_square_dual'}, inplace = True)
    train_data.drop(columns = ['time'], inplace = True)
    
    empty_train_col = [0]*len(train_data)
    for i, item in enumerate(test_data.columns):
        if item not in train_data.columns:
            train_data.insert(i, item, empty_train_col)

    empty_test_col = [0]*len(test_data)
    for i, item in enumerate(train_data.columns):
        if item not in test_data.columns:
            test_data.insert(i, item, empty_test_col)
    
    X_data = train_data.drop(columns = ['gl_predict'])
    y_data = train_data[['gl_predict']]
    input_dim = X_data.shape[1]
    
    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()
    scaler_x.fit(X_data)
    scaler_y.fit(y_data)
    scaled_X_train_data = scaler_x.transform(X_data)
    scaled_y_train_data = scaler_y.transform(y_data)
    scaled_X_train_data = np.reshape(scaled_X_train_data, (scaled_X_train_data.shape[0], 1, scaled_X_train_data.shape[1]))
    
    model = Sequential()
    model.add(Bidirectional(LSTM(128, input_shape = (scaled_X_train_data.shape[1], scaled_X_train_data.shape[2]))))
    model.add(Dense(150, activation = 'relu'))
    model.add(Dropout(0.20))
    model.add(Dense(100, activation = 'relu'))
    model.add(Dropout(0.15))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(20, activation = 'relu'))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse', optimizer = 'adam')
    model.fit(scaled_X_train_data, scaled_y_train_data, epochs = 200, batch_size = 32, shuffle = False)
    
    X_test_data = test_data.drop(columns = ['gl_predict'])
    y_test_data = test_data[['gl_predict']]
    scaled_X_test_data = scaler_x.transform(X_test_data)
    scaled_X_test_data = np.reshape(scaled_X_test_data, (scaled_X_test_data.shape[0], 1, scaled_X_test_data.shape[1]))
    prediction = model.predict(scaled_X_test_data, batch_size = 32)
    scaled_prediction = scaler_y.inverse_transform(prediction)
    
    final_output = pd.DataFrame(columns = ['timestamp', 'predicted_BGL_value'])
    final_output['timestamp'] = test_time
#     final_output['true_BGL_value'] = test_gl_value
    final_output['predicted_BGL_value'] = scaled_prediction
    
    return (final_output, math.sqrt(mean_squared_error(scaled_prediction, y_test_data)), mean_absolute_error(scaled_prediction, y_test_data))

Using TensorFlow backend.


time: 2.05 s


In [3]:
test_data_list = ['540-ws-training.csv', '544-ws-training.csv', '552-ws-training.csv', '567-ws-training.csv', '584-ws-training.csv', '596-ws-training.csv']

rmse_list = {}
mae_list = {}
for file in os.listdir('../data/derived/60_min_complete_dataset/'):
    if file in test_data_list:
        train_data = pd.read_csv('../data/derived/60_min_complete_dataset/'+file)
        test_data = pd.read_csv('../data/derived_test/60_min_complete_dataset/'+file[0:7]+'testing.csv')
        final_output, rmse_error, mae_error = model(test_data, train_data)
        rmse_list[file] = rmse_error
        mae_list[file] = mae_error
        print ('{} - {}'.format(file, rmse_error))
        print ('{} - {}'.format(file, mae_error))
        final_output.to_csv('../data/output/bidirectional_lstm/'+file[0:7]+'result_60min.csv')

Epoch 1/200
1103/1103 [==============================] - 2s 2ms/step - loss: 0.0539
Epoch 2/200
1103/1103 [==============================] - 0s 157us/step - loss: 0.0205
Epoch 3/200
1103/1103 [==============================] - 0s 156us/step - loss: 0.0158
Epoch 4/200
1103/1103 [==============================] - 0s 163us/step - loss: 0.0150
Epoch 5/200
1103/1103 [==============================] - 0s 167us/step - loss: 0.0142
Epoch 6/200
1103/1103 [==============================] - 0s 165us/step - loss: 0.0141
Epoch 7/200
1103/1103 [==============================] - 0s 184us/step - loss: 0.0140
Epoch 8/200
1103/1103 [==============================] - 0s 167us/step - loss: 0.0132
Epoch 9/200
1103/1103 [==============================] - 0s 160us/step - loss: 0.0138
Epoch 10/200
1103/1103 [==============================] - 0s 165us/step - loss: 0.0136
Epoch 11/200
1103/1103 [==============================] - 0s 161us/step - loss: 0.0132
Epoch 12/200
1103/1103 [==============================

1103/1103 [==============================] - 0s 174us/step - loss: 0.0103
Epoch 96/200
1103/1103 [==============================] - 0s 166us/step - loss: 0.0107
Epoch 97/200
1103/1103 [==============================] - 0s 159us/step - loss: 0.0098
Epoch 98/200
1103/1103 [==============================] - 0s 162us/step - loss: 0.0099
Epoch 99/200
1103/1103 [==============================] - 0s 154us/step - loss: 0.0101
Epoch 100/200
1103/1103 [==============================] - 0s 158us/step - loss: 0.0105
Epoch 101/200
1103/1103 [==============================] - 0s 161us/step - loss: 0.0107
Epoch 102/200
1103/1103 [==============================] - 0s 164us/step - loss: 0.0110
Epoch 103/200
1103/1103 [==============================] - 0s 219us/step - loss: 0.0100
Epoch 104/200
1103/1103 [==============================] - 0s 225us/step - loss: 0.0097
Epoch 105/200
1103/1103 [==============================] - 0s 172us/step - loss: 0.0099
Epoch 106/200
1103/1103 [=========================

1103/1103 [==============================] - 0s 236us/step - loss: 0.0092
Epoch 189/200
1103/1103 [==============================] - 0s 235us/step - loss: 0.0088
Epoch 190/200
1103/1103 [==============================] - 0s 219us/step - loss: 0.0088
Epoch 191/200
1103/1103 [==============================] - 0s 246us/step - loss: 0.0089
Epoch 192/200
1103/1103 [==============================] - 0s 230us/step - loss: 0.0090
Epoch 193/200
1103/1103 [==============================] - 0s 227us/step - loss: 0.0091
Epoch 194/200
1103/1103 [==============================] - 0s 242us/step - loss: 0.0083
Epoch 195/200
1103/1103 [==============================] - 0s 266us/step - loss: 0.0086
Epoch 196/200
1103/1103 [==============================] - 0s 247us/step - loss: 0.0086
Epoch 197/200
1103/1103 [==============================] - 0s 210us/step - loss: 0.0094
Epoch 198/200
1103/1103 [==============================] - 0s 220us/step - loss: 0.0088
Epoch 199/200
1103/1103 [=====================

1127/1127 [==============================] - 0s 227us/step - loss: 0.0091
Epoch 82/200
1127/1127 [==============================] - 0s 229us/step - loss: 0.0091
Epoch 83/200
1127/1127 [==============================] - 0s 214us/step - loss: 0.0089
Epoch 84/200
1127/1127 [==============================] - 0s 234us/step - loss: 0.0092
Epoch 85/200
1127/1127 [==============================] - 0s 236us/step - loss: 0.0090
Epoch 86/200
1127/1127 [==============================] - 0s 271us/step - loss: 0.0089
Epoch 87/200
1127/1127 [==============================] - 0s 233us/step - loss: 0.0086
Epoch 88/200
1127/1127 [==============================] - 0s 242us/step - loss: 0.0088
Epoch 89/200
1127/1127 [==============================] - 0s 227us/step - loss: 0.0089
Epoch 90/200
1127/1127 [==============================] - 0s 239us/step - loss: 0.0088
Epoch 91/200
1127/1127 [==============================] - 0s 231us/step - loss: 0.0089
Epoch 92/200
1127/1127 [==============================] 

1127/1127 [==============================] - 0s 229us/step - loss: 0.0078
Epoch 175/200
1127/1127 [==============================] - 0s 253us/step - loss: 0.0078
Epoch 176/200
1127/1127 [==============================] - 0s 242us/step - loss: 0.0078
Epoch 177/200
1127/1127 [==============================] - 0s 276us/step - loss: 0.0080
Epoch 178/200
1127/1127 [==============================] - 0s 239us/step - loss: 0.0081
Epoch 179/200
1127/1127 [==============================] - 0s 233us/step - loss: 0.0079
Epoch 180/200
1127/1127 [==============================] - 0s 213us/step - loss: 0.0082
Epoch 181/200
1127/1127 [==============================] - 0s 248us/step - loss: 0.0079
Epoch 182/200
1127/1127 [==============================] - 0s 250us/step - loss: 0.0081
Epoch 183/200
1127/1127 [==============================] - 0s 252us/step - loss: 0.0078
Epoch 184/200
1127/1127 [==============================] - 0s 240us/step - loss: 0.0076
Epoch 185/200
1127/1127 [=====================

...

924/924 [==============================] - 0s 274us/step - loss: 0.0115
Epoch 150/200
924/924 [==============================] - 0s 342us/step - loss: 0.0096
Epoch 151/200
924/924 [==============================] - 0s 289us/step - loss: 0.0089
Epoch 152/200
924/924 [==============================] - 0s 282us/step - loss: 0.0088
Epoch 153/200
924/924 [==============================] - 0s 275us/step - loss: 0.0099
Epoch 154/200
924/924 [==============================] - 0s 292us/step - loss: 0.0105
Epoch 155/200
924/924 [==============================] - 0s 288us/step - loss: 0.0106
Epoch 156/200
924/924 [==============================] - 0s 280us/step - loss: 0.0102
Epoch 157/200
924/924 [==============================] - 0s 293us/step - loss: 0.0094
Epoch 158/200
924/924 [==============================] - 0s 277us/step - loss: 0.0097
Epoch 159/200
924/924 [==============================] - 0s 326us/step - loss: 0.0095
Epoch 160/200
924/924 [==============================] - 0s 266us/st

1055/1055 [==============================] - 0s 333us/step - loss: 0.0080
Epoch 43/200
1055/1055 [==============================] - 0s 328us/step - loss: 0.0083
Epoch 44/200
1055/1055 [==============================] - 0s 385us/step - loss: 0.0077
Epoch 45/200
1055/1055 [==============================] - 0s 328us/step - loss: 0.0083
Epoch 46/200
1055/1055 [==============================] - 0s 358us/step - loss: 0.0080
Epoch 47/200
1055/1055 [==============================] - 0s 315us/step - loss: 0.0076
Epoch 48/200
1055/1055 [==============================] - 0s 394us/step - loss: 0.0076
Epoch 49/200
1055/1055 [==============================] - 0s 426us/step - loss: 0.0080
Epoch 50/200
1055/1055 [==============================] - 0s 383us/step - loss: 0.0078
Epoch 51/200
1055/1055 [==============================] - 0s 430us/step - loss: 0.0079
Epoch 52/200
1055/1055 [==============================] - 0s 449us/step - loss: 0.0076 0s - loss: 0.009 - ETA: 0s - loss: 0.009 - ETA: 0s - los

1055/1055 [==============================] - 0s 287us/step - loss: 0.0065
Epoch 135/200
1055/1055 [==============================] - 0s 301us/step - loss: 0.0066
Epoch 136/200
1055/1055 [==============================] - 0s 397us/step - loss: 0.0069
Epoch 137/200
1055/1055 [==============================] - 0s 376us/step - loss: 0.0066
Epoch 138/200
1055/1055 [==============================] - 0s 266us/step - loss: 0.0064
Epoch 139/200
1055/1055 [==============================] - 0s 319us/step - loss: 0.0067
Epoch 140/200
1055/1055 [==============================] - 0s 409us/step - loss: 0.0065
Epoch 141/200
1055/1055 [==============================] - 0s 401us/step - loss: 0.0066
Epoch 142/200
1055/1055 [==============================] - 0s 368us/step - loss: 0.0067
Epoch 143/200
1055/1055 [==============================] - 0s 471us/step - loss: 0.0066
Epoch 144/200
1055/1055 [==============================] - 0s 422us/step - loss: 0.0064
Epoch 145/200
1055/1055 [=====================

1092/1092 [==============================] - 0s 383us/step - loss: 0.0131
Epoch 27/200
1092/1092 [==============================] - 0s 407us/step - loss: 0.0129
Epoch 28/200
1092/1092 [==============================] - 0s 307us/step - loss: 0.0124
Epoch 29/200
1092/1092 [==============================] - 1s 472us/step - loss: 0.0123
Epoch 30/200
1092/1092 [==============================] - 0s 408us/step - loss: 0.0124
Epoch 31/200
1092/1092 [==============================] - 0s 375us/step - loss: 0.0127
Epoch 32/200
1092/1092 [==============================] - 0s 413us/step - loss: 0.0128
Epoch 33/200
1092/1092 [==============================] - 0s 381us/step - loss: 0.0123
Epoch 34/200
1092/1092 [==============================] - 0s 361us/step - loss: 0.0119
Epoch 35/200
1092/1092 [==============================] - 0s 435us/step - loss: 0.0122
Epoch 36/200
1092/1092 [==============================] - 1s 513us/step - loss: 0.0123
Epoch 37/200
1092/1092 [==============================] 

1092/1092 [==============================] - 0s 407us/step - loss: 0.0100
Epoch 120/200
1092/1092 [==============================] - 0s 376us/step - loss: 0.0095
Epoch 121/200
1092/1092 [==============================] - 0s 384us/step - loss: 0.0100
Epoch 122/200
1092/1092 [==============================] - 0s 421us/step - loss: 0.0095
Epoch 123/200
1092/1092 [==============================] - 1s 575us/step - loss: 0.0094
Epoch 124/200
1092/1092 [==============================] - 1s 535us/step - loss: 0.0092
Epoch 125/200
1092/1092 [==============================] - 0s 391us/step - loss: 0.0101
Epoch 126/200
1092/1092 [==============================] - 0s 407us/step - loss: 0.0097
Epoch 127/200
1092/1092 [==============================] - 0s 340us/step - loss: 0.0094
Epoch 128/200
1092/1092 [==============================] - 0s 347us/step - loss: 0.0095
Epoch 129/200
1092/1092 [==============================] - 0s 452us/step - loss: 0.0091
Epoch 130/200
1092/1092 [=====================

In [4]:
rmse_list

{'584-ws-training.csv': 41.49646610425353,
 '567-ws-training.csv': 37.62514174447541,
 '596-ws-training.csv': 27.82456919855868,
 '552-ws-training.csv': 39.12656596297527,
 '544-ws-training.csv': 29.135643762238626,
 '540-ws-training.csv': 48.70302178113067}

time: 8.34 ms


In [5]:
mae_list

{'584-ws-training.csv': 31.420770168833347,
 '567-ws-training.csv': 27.345672061318133,
 '596-ws-training.csv': 20.654920638311474,
 '552-ws-training.csv': 29.28454083773329,
 '544-ws-training.csv': 21.03589979380137,
 '540-ws-training.csv': 36.55271200461275}

time: 8.43 ms


In [6]:
from statistics import pstdev, mean

rmse_values = rmse_list.values()
mae_values = mae_list.values()

print ('Mean - RMSE : {}'.format(mean(rmse_values)))
print ('Standard Deviation - RMSE : {}'.format(pstdev(rmse_values)))

print ('Mean - MAE : {}'.format(mean(mae_values)))
print ('Standard Deviation - MAE : {}'.format(pstdev(mae_values)))

Mean - RMSE : 37.31856809227203
Standard Deviation - RMSE : 7.159140158821004
Mean - MAE : 27.71575258410173
Standard Deviation - MAE : 5.61089482387882
time: 17.8 ms
